In [1]:
import requests
import json
import pandas as pd
import numpy as np
import geojson
from shapely.geometry import mapping, shape, MultiPoint

In [2]:
with open("data/daerah.geojson") as file:
    countys = json.load(file)

In [3]:
countys = {malaysia['properties']['daerah']:malaysia['geometry'] for malaysia in countys['features']}
countys.keys()

dict_keys(['Batu Pahat', 'Johor Bahru', 'Kluang', 'Kota Tinggi', 'Kulai', 'Tangkak', 'Mersing', 'Muar', 'Pontian', 'Segamat', 'Baling', 'Bandar Baharu', 'Kota Setar', 'Kuala Muda', 'Kubang Pasu', 'Kulim', 'Langkawi', 'Padang Terap', 'Pendang', 'Pokok Sena', 'Sik', 'Yan', 'Bachok', 'Gua Musang', 'Jeli', 'Kota Bharu', 'Kuala Krai', 'Machang', 'Pasir Mas', 'Pasir Puteh', 'Tanah Merah', 'Tumpat', 'Bandar Tun Razak', 'Batu', 'Bukit Bintang', 'Cheras', 'Kepong', 'Lembah Pantai', 'Segambut', 'Seputeh', 'Setiawangsa', 'Titiwangsa', 'Wangsa Maju', 'Labuan', 'Alor Gajah', 'Jasin', 'Melaka Tengah', 'Jelebu', 'Jempol', 'Kuala Pilah', 'Port Dickson', 'Rembau', 'Seremban', 'Tampin', 'Bentong', 'Bera', 'Cameron Highlands', 'Jerantut', 'Kuantan', 'Lipis', 'Maran', 'Pekan', 'Raub', 'Rompin', 'Temerloh', 'Batang Padang', 'Hilir Perak', 'Hulu Perak', 'Kampar', 'Kerian', 'Kinta', 'Kuala Kangsar', 'Larut Matang Selama', 'Manjung', 'Perak Tengah', 'Perlis', 'Barat Daya', 'Seberang Perai Selatan', 'Seberang 

In [4]:
# Check if generated points are in the provided polygon boundary
def check_intersection(df):
    points = MultiPoint(df.values)
    try:
        ls = list(points.intersection(polygon))
        return [[l.x, l.y] for l in ls]
    except:
        return []

point_my = dict()

for county_name in countys.keys():
    #print(county_name)
    county = json.dumps(countys[county_name])
    c_geojson = geojson.loads(county)
    polygon = shape(c_geojson)
    lonmin, latmin, lonmax, latmax = polygon.bounds
    resolution = 0.003
    lat_arange = np.arange(latmin, latmax, resolution)
    lon_arange = np.arange(lonmin, lonmax, resolution)
    x, y = np.meshgrid(lat_arange, lon_arange)
    x = np.round(x, 6)
    y = np.round(y, 6)
    x = np.reshape(x, (-1,1))
    y = np.reshape(y, (-1,1))
    xy = np.concatenate([y,x],axis=1)
    df = pd.DataFrame(xy)
    check_points = check_intersection(df)
    in_county = []
    for point in check_points:
        in_county.extend([point])

    point_my[county_name] = in_county

TopologyException: Input geom 1 is invalid: Self-intersection at or near point 101.72416454244099 3.2292779974886989 at 101.72416454244099 3.2292779974886989
TopologyException: Input geom 1 is invalid: Self-intersection at or near point 101.737602 3.1392709999999999 at 101.737602 3.1392709999999999


In [5]:
point_my.keys()

dict_keys(['Batu Pahat', 'Johor Bahru', 'Kluang', 'Kota Tinggi', 'Kulai', 'Tangkak', 'Mersing', 'Muar', 'Pontian', 'Segamat', 'Baling', 'Bandar Baharu', 'Kota Setar', 'Kuala Muda', 'Kubang Pasu', 'Kulim', 'Langkawi', 'Padang Terap', 'Pendang', 'Pokok Sena', 'Sik', 'Yan', 'Bachok', 'Gua Musang', 'Jeli', 'Kota Bharu', 'Kuala Krai', 'Machang', 'Pasir Mas', 'Pasir Puteh', 'Tanah Merah', 'Tumpat', 'Bandar Tun Razak', 'Batu', 'Bukit Bintang', 'Cheras', 'Kepong', 'Lembah Pantai', 'Segambut', 'Seputeh', 'Setiawangsa', 'Titiwangsa', 'Wangsa Maju', 'Labuan', 'Alor Gajah', 'Jasin', 'Melaka Tengah', 'Jelebu', 'Jempol', 'Kuala Pilah', 'Port Dickson', 'Rembau', 'Seremban', 'Tampin', 'Bentong', 'Bera', 'Cameron Highlands', 'Jerantut', 'Kuantan', 'Lipis', 'Maran', 'Pekan', 'Raub', 'Rompin', 'Temerloh', 'Batang Padang', 'Hilir Perak', 'Hulu Perak', 'Kampar', 'Kerian', 'Kinta', 'Kuala Kangsar', 'Larut Matang Selama', 'Manjung', 'Perak Tengah', 'Perlis', 'Barat Daya', 'Seberang Perai Selatan', 'Seberang 

In [6]:
with open('my_points.json', 'w') as ofile:
    json.dump(point_my, ofile)